In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sys import platform
is_linux = (platform == "linux" or platform == "linux2")

from glob import glob
from os import listdir, mkdir, getcwd, remove
from os.path import isfile, join, basename, isdir

from analysis.split_event_tree import split_event_tree,ttype_column
from analysis.calc import calc_nll_llr_dtf_delta, get_kinematics, filter_for_tf
from analysis.plot_matplotlib import plot_hist

import os; print(os.environ['PATH']); print(os.environ['LD_LIBRARY_PATH'])
#from analysis.cffi.Physsim.CalculateME import lib as physsim
from analysis.cffi.mg5.lib import calc_zhh, calc_zzh
from tqdm.auto import tqdm

/nfs/dust/ilc/user/bliewert/.mambaforge/envs/py311/bin:/nfs/dust/ilc/user/bliewert/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/nfs/dust/ilc/user/bliewert/.mambaforge/envs/py311/bin:/nfs/dust/ilc/user/bliewert/.mambaforge/condabin:/usr/lib/heimdal/bin:/afs/desy.de/common/passwd:/usr/local/bin:/usr/bin:/afs/desy.de/user/b/bliewert/bin:/usr/local/sbin:/usr/sbin:/opt/puppetlabs/bin
/afs/desy.de/user/b/bliewert/public/ILCSoft/LCIO/lib64:/afs/desy.de/user/b/bliewert/public/ILCSoft/Physsim/lib64:/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/MEM_HEP/analysis/cffi/mg5/mg5/lib


/nfs/dust/ilc/user/bliewert/.mambaforge/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from analysis.save_data import pd_to_np
from analysis.import_data import import_data,filter_data,combine_columns

mcp_raw = import_data("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/mcparticle/compare_mcparticle.npy")
reco_raw = import_data("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/reco/compare_reco.npy")

recalculate = False

if recalculate:
    #tjmr_raw = import_data("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/truejet_matchingreco/compare_truejet_matchingreco.npy")
    #tjt_raw = import_data("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/truejet_true/compare_truejet_true.npy")
    #tjs_raw = import_data("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/truejet_seen/compare_truejet_seen.npy")

    reco = filter_for_tf(reco_raw)
    mcp = filter_for_tf(mcp_raw, jet=False, lepton=False)
        
    for run_type in ['reco', 'mcparticle']:
        data = mcp if run_type == 'mcparticle' else reco
        
        kinematics = []
        pbar = tqdm(range(len(data)))
        for i in pbar:
            pbar.set_description(f"{run_type}: Getting kinematics for event {i}")
            for value in get_kinematics(data, run_type == 'mcparticle', i): 
                kinematics.append(value)
            
        mg5_zhh = calc_zhh(kinematics)
        mg5_zzh = calc_zzh(kinematics)

        data["zhh_mg5"] = np.array(mg5_zhh)
        data["zzh_mg5"] = np.array(mg5_zzh)
        data["llr_mg5"] = np.log(data["zhh_mg5"]/data["zzh_mg5"])

        np.save(f"/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/{run_type}/compare_{run_type}_with_mg5.npy", pd_to_np(data))
        
else:
    mcp  = pd.DataFrame(np.load("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/mcparticle/compare_mcparticle_with_mg5.npy", allow_pickle=True))
    reco = pd.DataFrame(np.load("/nfs/dust/ilc/user/bliewert/fullflow_v3/comparison/npy/reco/compare_reco_with_mg5.npy", allow_pickle=True))
    
from analysis.plot_routines import plot_llr
from analysis.calc import calc_nll_llr_dtf_delta

mcp = calc_nll_llr_dtf_delta(filter_data(mcp))
reco = calc_nll_llr_dtf_delta(filter_data(reco))